In [1]:
import pandas as pd
import functions as fn

In [2]:
tickers = pd.read_csv("./data/NASDAQ100Components")
tickers = tickers['Ticker'].tolist()
etf = 'QQQ'
start = '2018-11-01'
end = '2019-10-31'
cointegrationResultsIndividual = fn.runCointTestIndividual(etf, tickers, start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ATVI
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
ALXN
[*********************100%***********************]  1 of 1 completed
ALGN
[*********************100%***********************]  1 of 1 completed
GOOGL
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
AMGN
[*********************100%***********************]  1 of 1 completed
ADI
[*********************100%***********************]  1 of 1 completed
ANSS
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%******************

In [3]:
cointegrationResultsIndividual.head()

,ticker,t-stat,pval
53,JD,-3.801199,0.013506
73,NXPI,-3.702010,0.018217
10,ANSS,-3.680510,0.019410
60,LULU,-3.612945,0.023612
88,SNPS,-3.585750,0.025514


In [4]:
subset = cointegrationResultsIndividual.loc[cointegrationResultsIndividual['pval'] <= 0.1]
tickers = subset['ticker'].tolist()
etf = 'QQQ'
start = '2018-11-01'
end = '2019-10-31'
synthetic_asset_rank = fn.runCointTestBaskets(etf, tickers, start, end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  14 of 14 completed
0 done, out of a total of  16369
500 done, out of a total of  16369
1000 done, out of a total of  16369
1500 done, out of a total of  16369
2000 done, out of a total of  16369
2500 done, out of a total of  16369
3000 done, out of a total of  16369
3500 done, out of a total of  16369
4000 done, out of a total of  16369
4500 done, out of a total of  16369
5000 done, out of a total of  16369
5500 done, out of a total of  16369
6000 done, out of a total of  16369
6500 done, out of a total of  16369
7000 done, out of a total of  16369
7500 done, out of a total of  16369
8000 done, out of a total of  16369
8500 done, out of a total of  16369
9000 done, out of a total of  16369


KeyboardInterrupt: 

In [ ]:
syntheticAssetRankDf = synthetic_asset_rank[['ticker', 't-stat', 'pval']]
syntheticAssetRankDf.head()

In [ ]:
qqqRaw = pd.read_csv("./data/qqq.csv")
snpsRaw = pd.read_csv("./data/snps.csv")
orlyRaw = pd.read_csv("./data/orly.csv")
mdlzRaw = pd.read_csv("./data/mdlz.csv")

qqqRaw = qqqRaw[['DATE', 'TIME_M', 'PRICE', 'SIZE']]
snpsRaw = snpsRaw[['DATE', 'TIME_M', 'PRICE', 'SIZE']]
orlyRaw = orlyRaw[['DATE', 'TIME_M', 'PRICE', 'SIZE']]
mdlzRaw = mdlzRaw[['DATE', 'TIME_M', 'PRICE', 'SIZE']]

In [ ]:
qqqClean = fn.cleanTickData(qqqRaw)
snpsClean = fn.cleanTickData(snpsRaw)
orlyClean = fn.cleanTickData(orlyRaw)
mdlzClean = fn.cleanTickData(mdlzRaw)

In [ ]:
qqq = fn.getMinuteData(qqqClean)
snps = fn.getMinuteData(snpsClean)
orly = fn.getMinuteData(orlyClean)
mdlz = fn.getMinuteData(mdlzClean)

In [ ]:
qqq = qqq.set_index('date_time')
qqq = qqq.rename(columns={'close': 'qqqclose'})

snps = snps.set_index('date_time')
snps = snps.rename(columns={'close': 'snpsclose'})

orly = orly.set_index('date_time')
orly = orly.rename(columns={'close': 'orlyclose'})

mdlz = mdlz.set_index('date_time')
mdlz = mdlz.rename(columns={'close': 'mdlzclose'})

data = qqq[['qqqclose']].join([snps[['snpsclose']], orly[['orlyclose']], mdlz[['mdlzclose']]], how='outer')
data = data.ffill()
data.tail(20)

In [ ]:
syntheticAssetPrice = data['snpsclose'] + data['orlyclose'] + data['mdlzclose']
syntheticAssetLogPrice = np.log(syntheticAssetPrice.values)
qqqLogPrice = np.log(data['qqqclose'].values)

In [ ]:
kf = fn.univariateKalmanFilter(syntheticAssetLogPrice, qqqLogPrice)
state_means, state_covs = kf.filter(qqqLogPrice)
slope=state_means[:, 0] 
intercept=state_means[:, 1]

In [ ]:
plt.figure(figsize =(15,7))
plt.plot(slope, c='b')
plt.ylabel('slope')
plt.figure(figsize =(15,7))
plt.plot(intercept,c='r')
plt.ylabel('intercept')

In [ ]:
spread_ts = qqqLogPrice - slope * syntheticAssetLogPrice
plt.figure(figsize = (15,7))
plt.plot(spread_ts)
plt.title("Spread based on Dynamic Hedge Ratio")
plt.ylabel("Log Spread ($)")
plt.xlabel("Date")
spread_avg = spread_ts.mean()
spread_sd = spread_ts.std()
upper_threshold = spread_avg + spread_sd
lower_threshold = spread_avg - spread_sd
plt.axhline(upper_threshold, linestyle = 'dashed', color = 'g')
plt.axhline(spread_avg, linestyle = 'dashed', color = 'y')
plt.axhline(lower_threshold, linestyle = 'dashed', color = 'r')

In [ ]:
data.reset_index(level=0, inplace=True)
data['logspread'] = spread_ts
data['dayperiod'] = data['date_time'].dt.to_period('D')
data.tail()

In [ ]:
spread_df = data[['date_time', 'logspread', 'dayperiod']]
spread_df['hedgeRatio'] = slope

lookback = 180
spread_df['spread'] = np.exp(spread_df['logspread'])
spread_df['returns'] = spread_df['spread'].diff() / spread_df['spread'].abs().shift()
spread_df['ema'] = spread_df['spread'].ewm(span=lookback).mean()

constant = 1
rolling_std = spread_df['spread'].rolling(lookback).std()
spread_df['upperband'] = spread_df['ema'] + (constant * rolling_std)
spread_df['lowerband'] = spread_df['ema'] - (constant * rolling_std)

spread_df['aboveOrBelowEMA'] = np.where(spread_df['spread'] > spread_df['ema'], 1, -1)
spread_df = spread_df.iloc[lookback:]
spread_df

In [ ]:
spread_df['position'] = np.where(spread_df['spread'] > spread_df['upperband'], -1, np.nan)
spread_df['position'] = np.where(spread_df['spread'] < spread_df['lowerband'], 1, spread_df['position'])
spread_df['position'] = np.where(spread_df['aboveOrBelowEMA'] * spread_df['aboveOrBelowEMA'].shift(1) < 0, 
                                 0, spread_df['position'])
spread_df['position'] = spread_df['position'].ffill().fillna(0)